In [34]:
import os
import pickle
import pandas as pd

from bert_score import score as bert_score_fn
import pickle
import evaluate
import string

In [35]:
not_present_dic={"age": "no.","history":"no history was found.","parenchymal_distortion":"no.","nodules_echo_size":"unknown."}

In [36]:
model="biogptQA2e55e66no_copydescription8batchfinal"
questions=["age","history","parenchymal_distortion","nodules_echo_size"]
# questions=["age","tipo","tecnica","family","history","symtomatic",
#            "prosthesis","birads","density_mammo","calcifications_benign","density_echo","ganglio_mamo","lymph_benign","lymph_suspicious","parenchymal_distortion","simple_cyst","ductal_ectasia",
#           "nodules_echo_num","nodules_echo_description","nodules_echo_shape","nodules_echo_margin","nodules_echo_echogenicity","nodules_echo_location","nodules_echo_size","nodules_echo_known","nodules_echo_stable"]

In [37]:
"unknown" in "adsdunknowna"

True

In [38]:

model="biogptQA2e55e66no_copydescription8batchfinal"
questions=["age","history","parenchymal_distortion","nodules_echo_size"]

In [39]:
def truncate_before_keyword(text, keyword):
    """Truncate text before keyword (e.g., 'comparative study') and end with a period."""
    keyword_pos = text.lower().find(keyword.lower())
    if keyword_pos == -1:
        cleaned = text.strip()
    else:
        # Text before keyword
        partial = text[:keyword_pos]
        # Find last punctuation before keyword
        match = re.search(r'(.+?)[\.\,\;\n][^.,;\n]*$', partial)
        cleaned = match.group(1).strip() if match else partial.strip()
    
    # Ensure the result ends with a single period
    if not cleaned.endswith('.'):
        cleaned += '.'
    
    return cleaned

In [40]:
def calculate_acc(pred,truth,tipo,point=False):
    acc=0
    total=0
    for key,real in truth.items():
        if key in pred:
            total+=1
            predicted=pred[key]   
            
            if predicted.startswith(not_present_dic[tipo]):
                predicted="no response."
            elif tipo=="nodules_echo_size":
                predicted = predicted.split("mm")[0] + "mm."
            else:
                predicted = predicted.split(".")[0] + "."
            predicted=truncate_before_keyword(predicted, "comparative study")
            if predicted.lower()==str(real).lower():
                acc+=1
            # else:
            #     print(key,"REAL: ", real,"PREDICTED: ",predicted)
        else:
            print(key)
    print(acc)
    print(total)

    return acc/total

In [41]:
import re
import string
import evaluate

rouge = evaluate.load("rouge")
exact_match = evaluate.load("exact_match")

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        return text.translate(str.maketrans('', '', string.punctuation))
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = set(prediction_tokens) & set(ground_truth_tokens)
    num_same = len(common)
    if len(prediction_tokens) == 0 or len(ground_truth_tokens) == 0:
        return int(prediction_tokens == ground_truth_tokens)
    if num_same == 0:
        return 0
    precision = num_same / len(prediction_tokens)
    recall = num_same / len(ground_truth_tokens)
    return 2 * precision * recall / (precision + recall)

def calculate_metrics(pred, truth, tipo):
    preds, refs = [], []
    for key, real in truth.items():
        if key not in pred:
            continue
        predicted = pred[key]
        if tipo in not_present_dic:
            if predicted.startswith(not_present_dic[tipo]):
                predicted="no response."
            elif tipo=="nodules_echo_size":
                predicted = predicted.split("mm")[0] + "mm."
            else:
                predicted = predicted.split(".")[0] + "."
        
        predicted=truncate_before_keyword(predicted, "comparative study")
        preds.append(predicted)
        refs.append(real)
    # Compute metrics
    f1s, ems = [], []
    for p, r in zip(preds, refs):
        f1s.append(f1_score(p, r))
        ems.append(int(normalize_answer(p) == normalize_answer(r)))


    rouge_scores = rouge.compute(predictions=preds, references=refs)
    rouge_l = rouge_scores["rougeL"]

    P, R, F1 = bert_score_fn(preds, refs, lang="en", verbose=False, rescale_with_baseline=True)
    bert_f1 = F1.mean().item()
    bert_R = R.mean().item()

    print(f"Examples: {len(preds)}")
    print(f"Token F1: {sum(f1s)/len(f1s):.4f}")
    print(f"Exact Match: {sum(ems)/len(ems):.4f}")
    print(f"ROUGE-L: {rouge_l:.4f}")
    print(f"BERTScore F1: {bert_f1:.4f}")
    print(f"BERTScore R: {bert_R:.4f}")

    return {
        "token_f1": sum(f1s)/len(f1s),
        "exact_match": sum(ems)/len(ems),
        "rouge_l": rouge_l,
        "bert_score_f1": bert_f1,
        "bert_score_R": bert_R,
        "list_bert": F1,
    }

In [42]:
model="clinicalt57e57no_copydescription16batchfinaltokenized"
f1_total=[]
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    truth = {
    k: ("no response." if v in list(not_present_dic.values()) else v)
    for k, v in truth.items()
    }
    if tipo!="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point="mm"))
    else:
        print(calculate_acc(output,truth,tipo))

    result=calculate_metrics(output, truth, tipo)
    f1_bio=result["list_bert"]
    f1_total+=f1_bio

AGE
209
212
0.9858490566037735


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Examples: 212
Token F1: 0.9893
Exact Match: 0.9858
ROUGE-L: 0.9906
BERTScore F1: 0.9910
BERTScore R: 0.9939


NameError: name 'f1_total' is not defined

In [43]:
model_name="biogpt"+"1e57"+"no_copy"+"description"+"16batch"+"final"
f1_total=[]
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    truth = {
    k: ("no response." if v in list(not_present_dic.values()) else v)
    for k, v in truth.items()
    }
    if tipo!="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point="mm"))
    else:
        print(calculate_acc(output,truth,tipo))

    result=calculate_metrics(output, truth, tipo)
    f1_bio=result["list_bert"]
    f1_total+=f1_bio

AGE
209
212
0.9858490566037735


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Examples: 212
Token F1: 0.9893
Exact Match: 0.9858
ROUGE-L: 0.9906
BERTScore F1: 0.9910
BERTScore R: 0.9939
HISTORY
161
186
0.8655913978494624


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Examples: 186
Token F1: 0.8862
Exact Match: 0.8656
ROUGE-L: 0.8926
BERTScore F1: 0.9148
BERTScore R: 0.9189
PARENCHYMAL_DISTORTION
170
186
0.9139784946236559


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Examples: 186
Token F1: 0.9582
Exact Match: 0.9194
ROUGE-L: 0.9622
BERTScore F1: 0.9659
BERTScore R: 0.9656
NODULES_ECHO_SIZE
72
82
0.8780487804878049


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Examples: 82
Token F1: 0.9075
Exact Match: 0.8780
ROUGE-L: 0.9099
BERTScore F1: 0.9371
BERTScore R: 0.9326


In [44]:
[round(v.item(),4) for v in f1_total]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.2917,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4909,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.316,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1

In [5]:
model_name="biogpt"+"21e51e57"+"no_copy"+"description"+"16batch"+"final"
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model_name}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo!="nodules_echo_size":
        print(calculate_acc(output,truth,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,point="mm"))
    else:
        print(calculate_acc(output,truth))

AGE
181-536-521-20221217-101331_age 46. no.
0.9952830188679245
TIPO
076-969-379-20240229-090526.971_tipo nodal staging ultrasound report. normal control or revision report.
087-932-399-20240520-152923_tipo biopsy report. normal control or revision report.
172-442-193-20230724-114130.406_tipo biopsy report. normal control or revision report.
278-883-630-20240227-153842.710_tipo biopsy report. normal control or revision report.
286-892-882-20240401-153506_tipo biopsy report. normal control or revision report.
331-153-519-20230703-152749_tipo biopsy report. normal control or revision report.
654-622-062-20230529-154932_tipo biopsy report. nodal staging ultrasound report.
800-359-086-20230516-105151.599_tipo nodal staging ultrasound report. normal control or revision report.
0.9622641509433962
TECNICA
195-203-909-20230615-120646_tecnica only ultrasound study. mammography and ultrasound.
209-923-502-20230315-172153_tecnica only mammography study. mammography and ultrasound.
286-892-882-2024

In [5]:
model="biogpt"+"5e65e63"
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo=="history" or tipo=="family" or tipo=="symtomatic" or tipo=="density_echo" or tipo=="nodules_echo_echogenicity" or tipo=="nodules_echo_margin" or tipo=="nodules_echo_shape"  or tipo=="nodules_echo_location":
        print(calculate_acc(output,truth,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,point="mm"))
    else:
        print(calculate_acc(output,truth))

AGE
0.9952830188679245
TIPO
0.0
TECNICA
0.9764150943396226
FAMILY
0.9946236559139785
HISTORY
0.9032258064516129
SYMTOMATIC
0.946236559139785
PROSTHESIS
0.9946236559139785
BIRADS
0.9838709677419355
DENSITY_MAMMO
0.978494623655914
CALCIFICATIONS_BENIGN


FileNotFoundError: [Errno 2] No such file or directory: 'Generativos/results_dic_calcifications_benign/biogpt5e65e63.pkl'

In [10]:
model="biogpt1e55e63"
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo=="history" or tipo=="family" or tipo=="symtomatic" or tipo=="density_echo" or tipo=="nodules_echo_echogenicity" or tipo=="nodules_echo_margin" or tipo=="nodules_echo_shape"  or tipo=="nodules_echo_location":
        print(calculate_acc(output,truth,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,point="mm"))
    else:
        print(calculate_acc(output,truth))

AGE
0.9952830188679245
TIPO
0.9669811320754716
TECNICA
0.9858490566037735
FAMILY
0.9838709677419355
HISTORY
0.9139784946236559
SYMTOMATIC
0.9516129032258065
PROSTHESIS
0.989247311827957
BIRADS
0.9838709677419355
DENSITY_MAMMO
0.9623655913978495
DENSITY_ECHO
0.9354838709677419
LYMPH_SUSPICIOUS
0.9838709677419355
NODULES_ECHO_NUM
0.8225806451612904
NODULES_ECHO_SHAPE
0.926829268292683
NODULES_ECHO_MARGIN
0.8658536585365854
NODULES_ECHO_ECHOGENICITY
0.926829268292683
NODULES_ECHO_LOCATION
0.6585365853658537
NODULES_ECHO_SIZE
0.9146341463414634
NODULES_ECHO_KNOWN
0.7439024390243902
NODULES_ECHO_STABLE
0.7727272727272727


In [5]:
calculate_acc(biogpt_age,truth_age)

181-536-521-20221217-101331 46 no


0.9952830188679245

In [6]:
calculate_acc(biogpt_tipo,truth_tipo)

001-550-138-20231207-120430 no yes
286-892-882-20240401-153506 no yes
331-153-519-20230703-152749 no yes
534-723-539-20231013-165228.835 no yes


0.9811320754716981

In [7]:
calculate_acc(biogpt_tecnica,truth_tecnica)

049-148-620-20240419-163052.629 mammography and ultrasound only ultrasound study
153-234-011-20230824-150145.697 mammography and ultrasound only ultrasound study
154-571-251-20230928-113422 only ultrasound study mammography and ultrasound
181-536-521-20221217-101331 mammography and ultrasound only ultrasound study
195-203-909-20230615-120646 only ultrasound study mammography and ultrasound
209-923-502-20230315-172153 only mammography study mammography and ultrasound
227-053-436-20230714-113137 mammography and ultrasound only ultrasound study
286-892-882-20240401-153506 only ultrasound study only mammography study
800-359-086-20230522-154052 only mammography study mammography and ultrasound


0.9575471698113207